In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install automl_alex.
#!pip install -q -U automl_alex

In [2]:
import automl_alex
import sklearn
import pandas as pd
import time
from automl_alex import DataPrepare
from automl_alex import AutoML, AutoMLClassifier, AutoMLRegressor
print('AutoML-Alex version:', automl_alex.__version__)

/Users/alex/opt/anaconda3/envs/automl_py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: dlopen(/Users/alex/opt/anaconda3/envs/automl_py310/lib/python3.10/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: '/usr/local/opt/libomp/lib/libomp.dylib'
  Referenced from: '/Users/alex/opt/anaconda3/envs/automl_py310/lib/python3.10/site-packages/lightgbm/lib_lightgbm.so'
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

In [ ]:
RANDOM_SEED = 42

# Classifier

## Data

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
# https://www.openml.org/d/179
dataset = fetch_openml(data_id=179, as_frame=True)
dataset.target = dataset.target.astype('category').cat.codes
dataset.data.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country
0,2,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,1,0,2,United-States
1,3,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,0,United-States
2,2,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States
3,3,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,2,United-States
4,1,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,2,Cuba


In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, 
                                                    dataset.target,
                                                    test_size=0.25, 
                                                    random_state=RANDOM_SEED,)
X_train.shape, X_test.shape

((36631, 14), (12211, 14))

## AutoML

In [5]:
model = AutoMLClassifier(random_state=RANDOM_SEED,)
model.fit(X_train, y_train, timeout=900)

22:38:18 | > Start Fit Base Model
22:38:58 | ##################################################
22:38:58 | > Start Fit Models 2
22:38:58 | ##################################################
22:38:58 | ##################################################
22:38:59 | > Step 1: calc parameters and pruned score: get test 10 trials
22:42:20 |  One iteration ~ 20.1 sec
22:42:20 |  Possible iters ~ 36.0
22:42:20 | ! Not enough time to find the optimal parameters. 
                     Possible iters < 100. 
                     Please, Increase the 'timeout' parameter for normal optimization.
22:42:20 | --------------------------------------------------
22:42:20 |   Pruned Threshold Score: 0.9028
22:42:20 | ##################################################
22:42:20 | > Step 2: Full opt with Threshold Score Pruner
22:42:20 | ##################################################
22:42:21 | > Start optimization with the parameters:
22:42:21 | CV_Folds = 7
22:42:21 | Score_CV_Folds = 3
22:42:21 | Feat

In [7]:
predicts = model.predict(X_test)

In [8]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predicts),4))

Test AUC:  0.913


## Save & Load

In [9]:
model.save('AutoML_model_1')

Save model


In [5]:
model_new = AutoMLClassifier(random_state=RANDOM_SEED,)
model_new = model_new.load('AutoML_model_1')

23:05:46 | Load AutoML


In [6]:
predicts = model_new.predict(X_test)
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predicts),4))

Test AUC:  0.9134


# Regression

## Data

In [ ]:
# https://www.openml.org/d/543
dataset = fetch_openml(data_id=543, as_frame=True)

X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(dataset.data), 
                                                    pd.DataFrame(dataset.target), 
                                                    test_size=0.15, 
                                                    random_state=RANDOM_SEED,)

X_train.shape, X_test.shape

In [ ]:
X_train.head(5)

## AutoML

In [ ]:
model = AutoMLRegressor(random_state=RANDOM_SEED,)
model.fit(X_train, y_train, timeout=900, verbose=1)

In [ ]:
predicts = model.predict(X_test)
print('Test MSE: ', round(sklearn.metrics.mean_squared_error(y_test, predicts),4))